In [1]:
import deribit_data as dm
from option import Option
import numpy as np
from utils import *
from sklearn.metrics import mean_squared_error
import plotly.graph_objects as go   

In [2]:
data = dm.Scraper(currency='BTC')

In [3]:
raw_df = data.collect_data(max_workers = 15, save_csv = False)

Data Collected


In [4]:
#raw_df.head()

In [5]:
option = Option(raw_df, 'B')

In [6]:
#option.init_merton(m = 0.625038301534241, lam =0.0010591027301815744, v = 0.511097066109511,  )
option.init_merton(reset=True)

In [7]:
option.append_imp_vol_to_df()
option.append_loc_vol_to_df()

/Users/pierrebelamri/Documents/GitHub/deribit_scraping/option.py:172: RuntimeWarning: invalid value encountered in double_scalars
  return np.sqrt(num/denom)
/Users/pierrebelamri/Documents/GitHub/deribit_scraping/option.py:172: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(num/denom)
/Users/pierrebelamri/Documents/GitHub/deribit_scraping/option.py:172: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(num/denom)


In [8]:
option.append_BS_price()

In [9]:
option.df.dropna(inplace=True)
#option.df = option.df.drop(option.df[option.df.IV_moneyness >= .5*10**6].index)

In [10]:
#option.optimize_merton(tol = 1e-6, max_iter = 600)



In [11]:
option.append_Merton_price()

/Users/pierrebelamri/Documents/GitHub/deribit_scraping/option.py:172: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(num/denom)


In [24]:
x = option.df['S']/option.df['K']

#x = option.df['K']
y = option.df['_T']
z = option.df['L_VOL']
z2 = option.df['mark_iv']/100


fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z , mode='markers', name='Local vol')])
#fig = go.Figure(data=[go.Surface(x=x, y=y, z=z , name='Local vol')])
                      #go.Scatter3d(x=x, y=y, z=z2 , mode='markers', name='Implied vol deribit')])
"""
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z , mode='markers', name='Black&Scholes model Price'),
                      go.Scatter3d(x=x, y=y, z=z2, mode='markers', name='Real Market Price')])
"""

fig.update_scenes(xaxis_title_text='S/K', yaxis_title_text='Exp', zaxis_title_text='IV') 
fig.show()

In [30]:
#LINEAR INTERPOLATION
new_df = option.df[['moneyness', '_T', 'I_VOL_BS']]
new_df = new_df.pivot(index = '_T',columns='moneyness', values='I_VOL_BS')

new_df=new_df.interpolate(method='linear',axis=1,limit_direction='both')
z = new_df
x = list(new_df.columns)
y = list(new_df.index)


fig = go.Figure(data=[go.Surface(z=z, x=x, y=y)])
fig.update_layout(title='Implied Volatility', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.update_scenes(xaxis_title_text='S/K', yaxis_title_text='Exp', zaxis_title_text='IV') 
fig.show()

In [31]:
#LINEAR INTERPOLATION
new_df = option.df[['moneyness', '_T', 'L_VOL']]
new_df = new_df.pivot(index = '_T',columns='moneyness', values='L_VOL')

new_df=new_df.interpolate(method='linear',axis=1,limit_direction='both')
z = new_df
x = list(new_df.columns)
y = list(new_df.index)


fig = go.Figure(data=[go.Surface(z=z, x=x, y=y)])
fig.update_layout(title='Local Volatility', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))

fig.update_scenes(xaxis_title_text='S/K', yaxis_title_text='Exp', zaxis_title_text='Local Vol') 
fig.show()

In [32]:

new_df = option.df[['K', '_T', 'L_VOL']]
new_df = new_df.pivot(index = '_T',columns='K', values='L_VOL')

new_df=new_df.interpolate(method='spline',order=2,axis=1)
z = new_df
x = list(new_df.columns)
y = list(new_df.index)


fig = go.Figure(data=[go.Surface(z=z, x=x, y=y)])
fig.update_layout(title='xxx', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()


In [17]:
rms = mean_squared_error(option.df['MERTON_PRICE'], option.df['mark_price'], squared=False)
print('Root Mean Squared Error = {}%'.format(round(rms*100,3)))

Root Mean Squared Error = 254822.79%


In [18]:
option.df.dropna(inplace=True)
option.df = option.df.drop(option.df[option.df.BS_PRICE >= .5].index)

In [41]:
x = option.df['K']
y = option.df['_T']
z = option.df['BS_PRICE']
z2 = option.df['mid']
z3 = option.df['MERTON_PRICE']


#fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z , mode='markers', name='BS price')])

fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z , mode='markers', name='Black&Scholes model Price', marker=dict(opacity=0.8)),
                      go.Scatter3d(x=x, y=y, z=z2, mode='markers', name='Real Market Price', marker=dict(opacity=0.8)),
                      go.Scatter3d(x=x, y=y, z=z3, mode='markers', name='Merton Model Price', marker=dict(opacity=0.8))])


fig.update_scenes(xaxis_title_text='Strike', yaxis_title_text='Exp', zaxis_title_text='Price') 
fig.show()

rms_merton = mean_squared_error(option.df['mid'], option.df['MERTON_PRICE'], squared=False)
rms_bs = mean_squared_error(option.df['mid'], option.df['BS_PRICE'], squared=False)

print('Root Mean Squared Error | MERTON = {}%'.format(round(rms_merton*100,6)))
print('Root Mean Squared Error | B&S    = {}%'.format(round(rms_bs*100,6)))

Root Mean Squared Error | MERTON = 1.260422%
Root Mean Squared Error | B&S    = 2.972109%
